In [1]:
import numpy as np 
import pandas as pd

In [2]:
movies = pd.read_csv("movies_data_5000.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/movies_data_5000.csv'

In [ ]:
movies.head()

: 

In [ ]:
# keep: genres, id, keywords, overview, production companies, title,vote_average,cast , crew

: 

In [ ]:
movies=movies[["id","title","overview","genres","keywords","cast","crew"]]

: 

In [ ]:
movies.dropna(inplace=True)

: 

In [ ]:
movies.head(1)

: 

In [ ]:
movies["genres"][0]

: 

In [ ]:
def extract(a):
    import ast
    L=[]
   
    for i in ast.literal_eval(a):
        L.append(i["name"])
    return L
        

: 

In [ ]:
movies["genres"]=movies["genres"].apply(extract)

: 

In [ ]:
movies.head(1)

: 

In [ ]:
movies["keywords"]=movies["keywords"].apply(extract)

: 

In [ ]:
movies.head(1)

: 

In [ ]:
def extract_cast(a):
    import ast
    L=[]
    c=0
    for i in ast.literal_eval(a):
        if(c!=3):
            L.append(i["name"])
            c+=1
        else:
            break
    return L
            

: 

In [ ]:
movies["cast"]=movies["cast"].apply(extract_cast)

: 

In [ ]:
movies.head(1)

: 

In [ ]:
def extract_crew(a):
    import ast
    L=[]
    
    for i in ast.literal_eval(a):
        if(i["job"]=="Director"):
            L.append(i["name"])
        
    return L

: 

In [ ]:
movies["crew"]=movies["crew"].apply(extract_crew)

: 

In [ ]:
movies.head()

: 

In [ ]:
movies["overview"]=movies["overview"].apply(lambda x: x.split())

: 

In [ ]:
movies.head(1)

: 

In [ ]:
def remove_space(l):
    L=[]
    for i in l:
        L.append(i.replace(' ',''))
    return L

: 

In [ ]:
 
movies["genres"]=movies["genres"].apply(lambda x: [j.replace(" ","") for j in x])
movies["keywords"]=movies["keywords"].apply(lambda x: [j.replace(" ","") for j in x])
movies["cast"]=movies["cast"].apply(lambda x: [j.replace(" ","") for j in x])
movies["crew"]=movies["crew"].apply(lambda x: [j.replace(" ","") for j in x])

: 

In [ ]:
movies["tags"]=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

: 

In [ ]:
movies.head()

: 

In [ ]:
new_df=movies[["id","title","tags"]]

: 

In [ ]:
new_df["tags"]=new_df["tags"].apply(lambda x: " ".join(x))

: 

In [ ]:
new_df["tags"]=new_df["tags"].apply(lambda x: x.lower())

: 

cleaning complete


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

: 

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df["tags"] =new_df["tags"].apply(stem)

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")

: 

In [ ]:
vectors=cv.fit_transform(new_df["tags"]).toarray()

: 

In [ ]:
vectors.shape

: 

In [ ]:
cv.get_feature_names_out()

: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

: 

In [ ]:
sim=cosine_similarity(vectors)

: 

In [ ]:
sim.shape

: 

In [ ]:
sim[0]

: 

In [ ]:
new_df.head(1)

: 

In [ ]:
def recommend(movie):
    m_index=new_df[new_df["title"]==movie].index[0]
    movies_list=sorted(list(enumerate(sim[m_index])),reverse=True,key=lambda x:x[1])[1:6]
    final=[]
    for i in movies_list:
        final.append(new_df.iloc[i[0]]["title"])
    return final
        

: 

In [ ]:
recommend("Avatar")

: 

In [ ]:
import pickle

: 

In [ ]:
pickle.dump(new_df,open("data.pkl","wb"))
pickle.dump(sim,open("sim.pkl","wb"))

: 